In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os
import sys
import os

# Adjust this to point to the project root (where 'src/' exists)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # one level up from notebooks/
sys.path.append(project_root)

# Now import works
from src.data.data_loader import get_image_dataloaders# or use test dataset manually


In [ ]:
num_classes = 101 

In [ ]:
class CustomResNet(nn.Module):
    def __init__(self, base_model, num_classes, dropout_prob):
        super().__init__()
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(base_model.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [ ]:
# Parameters (match config.yaml used during training)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # one level up
model_path = os.path.join(project_root, 'models', 'model_v1', 'best.pth')
print(model_path) 
dropout_prob = 0.5  # same as used during training
num_classes = 101   # <-- fix here

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Build model
base_model = models.resnet18(weights=None)
model = CustomResNet(base_model, num_classes, dropout_prob)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()


In [ ]:
_, test_loader, _ = get_image_dataloaders(
    data_dir=os.path.join(project_root, 'Data', '1', 'test'),
    batch_size=128,
    img_size=224,
    num_workers=4,
    val_split=False
)


In [ ]:
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct / total:.4f}")


In [ ]:
import os
import random
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import torch

from torchvision.datasets import ImageFolder

root_dir = os.path.join(project_root, 'Data', '1', 'test')  # change this to your image folder path
# Get class mapping
temp_dataset = ImageFolder(root=root_dir)
class_to_idx = temp_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

# ---- PARAMETERS ----
root_dir = os.path.join(project_root, 'Data', '1', 'test') # change this to your image folder path
num_images = 10
img_size = 224

# ---- TRANSFORMS ----
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

# ---- GET IMAGE PATHS WITH LABELS ----
image_paths = []
class_labels = []

for class_name in os.listdir(root_dir):
    class_dir = os.path.join(root_dir, class_name)
    if not os.path.isdir(class_dir):
        continue
    for fname in os.listdir(class_dir):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            image_paths.append(os.path.join(class_dir, fname))
            class_labels.append(class_name)

# ---- SAMPLE RANDOM 10 ----
samples = random.sample(list(zip(image_paths, class_labels)), num_images)

# ---- PREDICT & PLOT ----
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

for idx, (img_path, actual_class) in enumerate(samples):
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        pred_idx = output.argmax(dim=1).item()

    predicted_class = list(class_to_idx.keys())[list(class_to_idx.values()).index(pred_idx)]

    axes[idx].imshow(img)
    axes[idx].axis('off')
    axes[idx].set_title(f"Pred: {predicted_class}\nActual: {actual_class}", fontsize=12)

plt.tight_layout()
plt.show()